In [1]:
from pprint import pprint

from op_analytics.coreutils.partitioned.location import DataLocation
from op_analytics.coreutils.partitioned.reader import DataReader
from op_analytics.datapipeline.etl.ingestion.reader.byblock import construct_readers_byblock
from op_analytics.datapipeline.etl.ingestion.reader.request import BlockBatchRequest
from op_analytics.datapipeline.models.compute.modelspec import ModelsDataSpec
from op_analytics.datapipeline.models.compute.testutils import setup_execution_context

In [2]:
model_name = "refined_traces"


# Prepare data raeders
data_spec = ModelsDataSpec(models=[model_name],
    root_path_prefix="blockbatch")
blockbatch_request = BlockBatchRequest.build(
    chains=["base"],
    range_spec="@20250720:+1",
    root_paths_to_read=data_spec.input_root_paths,
)
readers: list[DataReader] = construct_readers_byblock(
    blockbatch_request=blockbatch_request,
    read_from=DataLocation.GCS,
)


# Show details for the batch we are processing.
pprint(readers[0])

# Set up execution context and get handles to model input args.
# In subsequent cells you can use the model input args however you want.
ctx, input_datasets, auxiliary_templates = setup_execution_context(
    model_name=model_name,
    data_reader=readers[0],  # use the first reader
)

2025-07-21 18:16:47 [debug    ] connecting to GOLDSKY Clickhouse client... counter=001/001 eta=None filename=client.py lineno=56 process=65921
2025-07-21 18:16:47 [info     ] loaded vault from .env file    counter=001/001 eta=None filename=vault.py lineno=32 process=65921
2025-07-21 18:16:47 [debug    ] loaded vault: 28 items         counter=001/001 eta=None filename=vault.py lineno=79 process=65921
2025-07-21 18:16:48 [debug    ] initialized GOLDSKY Clickhouse client. counter=001/001 eta=None filename=client.py lineno=61 process=65921
2025-07-21 18:16:48 [info     ] querying markers at time range min: 2025-07-20 00:00:00, max: 2025-07-21 00:00:00 root_paths=['ingestion/blocks_v1', 'ingestion/traces_v1', 'ingestion/transactions_v1'] filename=request.py lineno=146 process=65921
2025-07-21 18:16:48 [debug    ] connecting to OPLABS Clickhouse client... filename=client.py lineno=56 process=65921
2025-07-21 18:16:48 [debug    ] initialized OPLABS Clickhouse client. filename=client.py lineno

In [3]:
refined_txs_table = auxiliary_templates["refined_transactions_fees"].create_table(
    duckdb_context=ctx,
    template_parameters={
        "raw_blocks": input_datasets["ingestion/blocks_v1"].as_subquery(),
        "raw_transactions": input_datasets["ingestion/transactions_v1"].as_subquery(),
         "extra_cols": []  # or provide actual column expressions as strings
    }
)

2025-07-21 18:16:48 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=291 process=65921
2025-07-21 18:16:48 [info     ] constructed read_parquet() string with 1 paths filename=client.py lineno=291 process=65921
2025-07-21 18:16:48 [info     ] Rendering query                filename=querybuilder.py lineno=40 process=65921 template=refined_transactions_fees
2025-07-21 18:16:53 [info     ] duck db size: 12.3KB           filename=client.py lineno=40 process=65921


In [6]:
# check the difference between the calculated fees and the actual fees
ctx.client.sql("""
SELECT 
    (effective_l2_priority_fee_per_gas + base_fee_per_gas + legacy_extra_fee_per_gas) - gas_price AS diff,
    ( l1_fee + l2_base_fee+l2_priority_fee+l2_legacy_extra_fee) - tx_fee AS diff_2,
    *
FROM refined_transactions_fees
WHERE 
    gas_price > 0
    AND transaction_index > 0
    AND (effective_l2_priority_fee_per_gas + base_fee_per_gas + legacy_extra_fee_per_gas) - gas_price != 0
    -- AND ( l1_fee + l2_base_fee+l2_priority_fee+l2_legacy_extra_fee) - tx_fee != 0
""").show(max_rows=100)

┌───────┬────────┬──────┬─────────┬──────────┬─────────┬───────┬───────────────────┬──────────────┬────────────┬──────────────┬─────────────────┬─────────┬──────────────────┬───────────┬───────────┬─────────────┬─────────────────────┬────────┬──────────────┬──────────────────┬──────────────────┬──────────────────────────┬───────────────────────────────────┬───────────────┬────────────────────┬─────────────────────────┬──────────────────────────┬────────┬─────────────────┬─────────────┬───────────┬─────────┬───────────────────┬──────────────────┬───────────────────────┬─────────────────────────────────────┬────────────┬────────────────┬─────────────────────┬────────┬─────────────────────┬────────────────┬─────────────────────┬────────────────┬─────────────────────┬────────────────┬────────────────┬────────────────┬────────────────────┬────────────────────┬────────────────────┬────────────────────────┬────────────────────────────┬───────────────────┬────────────────────────┬─────────────